# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra
using ASEconvert

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al; psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]
    unit_cell = periodic_system(lattice, atoms, positions)

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE, make the supercell
    # and then convert back ...
    supercell_ase = convert_ase(unit_cell) * pytuple((repeat, 1, 1))
    supercell     = pyconvert(AbstractSystem, supercell_ase)

    # Unfortunately right now the conversion to ASE drops the pseudopotential information,
    # so we need to reattach it:
    supercell = attach_psp(supercell; Al="hgh/lda/al-q3")

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(supercell; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

    CondaPkg Found dependencies: /home/runner/.julia/packages/ASEconvert/CNQ1A/CondaPkg.toml
    CondaPkg Found dependencies: /home/runner/.julia/packages/PythonCall/qTEA1/CondaPkg.toml
    CondaPkg Dependencies already up to date


As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
convert_ase(periodic_system(setup.model)).write("al_supercell.png")

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123


Python: None

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
self_consistent_field(aluminium_setup(1); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298278052420                   -0.85    5.0         
  2   -8.300191175445       -2.72       -1.26    1.0    116ms
  3   -8.300433383138       -3.62       -1.89    3.2    338ms
  4   -8.300461441278       -4.55       -2.76    1.8    115ms
  5   -8.300464039279       -5.59       -3.08    3.0    155ms
  6   -8.300464341935       -6.52       -3.28    6.0    182ms
  7   -8.300464490016       -6.83       -3.43    6.0    213ms
  8   -8.300464570049       -7.10       -3.57    1.5    144ms
  9   -8.300464621010       -7.29       -3.76    3.0    216ms
 10   -8.300464635887       -7.83       -3.92    2.4    161ms
 11   -8.300464643638       -8.11       -4.29    2.2    165ms


In [4]:
self_consistent_field(aluminium_setup(2); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -16.67436132182                   -0.70    5.6         
  2   -16.67864337640       -2.37       -1.14    1.2    281ms
  3   -16.67918870843       -3.26       -1.87    2.9    323ms
  4   -16.67927750168       -4.05       -2.75    3.5    357ms
  5   -16.67928585580       -5.08       -3.15    5.4    434ms
  6   -16.67928619177       -6.47       -3.45    2.1    310ms
  7   -16.67928621577       -7.62       -3.92    2.2    299ms
  8   -16.67928622141       -8.25       -4.55    3.0    336ms


In [5]:
self_consistent_field(aluminium_setup(4); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32584169732                   -0.56    6.6         
  2   -33.33262685190       -2.17       -1.00    1.0    1.03s
  3   -33.33410559316       -2.83       -1.73    5.2    1.37s
  4   -33.33591883099       -2.74       -2.25    2.2    1.19s
  5   -33.33691029153       -3.00       -2.47   11.0    1.79s
  6   -33.33693326140       -4.64       -2.77    3.0    1.11s
  7   -33.33694364951       -4.98       -3.69    4.4    1.33s
  8   -33.33694371004       -7.22       -3.83    5.6    1.50s
  9   -33.33694376643       -7.25       -4.22    1.4    947ms


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298272241344                   -0.85    5.1         
  2   -8.300244124951       -2.71       -1.59    1.0   83.5ms
  3   -8.300435969538       -3.72       -2.62    2.6    141ms
  4   -8.300415091486   +   -4.68       -2.40    9.5    245ms
  5   -8.300464325995       -4.31       -3.45    1.0   90.8ms
  6   -8.300464595299       -6.57       -3.88    3.6    155ms
  7   -8.300464640942       -7.34       -4.40    1.5    116ms


In [7]:
self_consistent_field(aluminium_setup(4); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32550389127                   -0.56    6.0         
  2   -33.30415386765   +   -1.67       -1.26    1.8    884ms
  3   -4.864580525264   +    1.45       -0.34    6.9    1.95s
  4   -33.33468944202        1.45       -1.94    6.5    1.81s
  5   -33.24414644937   +   -1.04       -1.40    4.4    1.66s
  6   -33.12444261546   +   -0.92       -1.39    4.1    1.51s
  7   -33.19679852867       -1.14       -1.48    5.5    1.55s
  8   -33.33549696218       -0.86       -2.24    3.9    1.34s
  9   -33.33581596609       -3.50       -2.38    2.2    1.13s
 10   -33.33677976848       -3.02       -2.76    1.9    1.09s
 11   -33.33685112383       -4.15       -3.01    2.1    997ms
 12   -33.33694264382       -4.04      

For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.